In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
from openai import OpenAI
import os
import re
import pandas as pd
import time
from tqdm import tqdm
import requests
from datasets import load_dataset
import pandas as pd

pd.options.display.max_rows = 999


import warnings
warnings.filterwarnings("ignore")


In [ ]:
class Evaluator:
    EVALUATION_PROMPT_TEMPLATE = """

        You are a helpful assistant assigned to evaluate the quality of a bot's response in the context of a user question. Your goal is to assign a numeric score from 1 to 5 based on clarity, correctness, and completeness. Please, if needed you use decimals to score.

        ---

        **Your Evaluation Task:**
        You are given:
        - A question from a user
        - A bot-generated response

        You must consider the quality of the response **in direct relation to the question**. Be strict but fair.

        ---

        **Scoring Guidelines:**

        - **5** → Fully correct, clearly written, and covers all critical information.
        - **4** → Mostly correct and complete, but with minor issues in clarity or missing minor details.
        - **3** → Understandable and mostly accurate, but misses important elements or has structural issues.
        - **2** → Contains factual inaccuracies or lacks important points; hard to follow.
        - **1** → Unclear, incorrect, or irrelevant.

        ---

        **Important:** You may use **decimal values** like 4.5, 3.0, or 2.75 if needed for more nuanced evaluation.

        Do **not** write any explanation—only return the numeric score.

        ---

        **Question:**
        {document}

        **Bot Response:**
        {summary}

        ---

        **Score:**
        """


    def __init__(self, client):
        self.client = client

    def get_geval_score(self,expected_response_value: str, bot_response: str) -> str:
        prompt = self.EVALUATION_PROMPT_TEMPLATE.format(

            document=expected_response_value,
            summary=bot_response,

        )
        response = self.client.chat.completions.create(
            model= "gpt-4-turbo" 
            messages=[{"role": "user", "content": prompt}],
            seed = 42,
            temperature=0.001,
            top_p=1,

        )

        score = response.choices[0].message.content.strip()


        return score




    def evaluate(self, expected_response_value: str, bot_response: str) -> str:
        return self.get_geval_score(expected_response_value, bot_response)



OpenAI GPT Evaluator

In [ ]:
#from evaluation import Evaluator

In [ ]:
### OPEN AI KEY

In [ ]:
evaluator = Evaluator(key)

In [7]:

# Cargar el dataset UltraFeedback (full con score)
dataset = load_dataset("argilla/ultrafeedback-binarized-preferences", split="train")

# Convertir a DataFrame
df = pd.DataFrame(dataset)


print(df.head())


README.md:   0%|          | 0.00/8.62k [00:00<?, ?B/s]

(…)-00000-of-00001-9dffc9d46d32c335.parquet:   0%|          | 0.00/110M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/63619 [00:00<?, ? examples/s]

          source                                        instruction  \
0  evol_instruct  Can you write a C++ program that prompts the u...   
1  evol_instruct  Suppose you are a content creator and want to ...   
2  evol_instruct  Identify the interrelated economic, political,...   
3  evol_instruct  How can I convert the decimal number 31 to bin...   
4  evol_instruct  Can you modify the C++ code provided below to ...   

                                     chosen_response  \
0  Here's a C++ program that prompts the user to ...   
1  To use GPT for generating compelling titles an...   
2  The stock market crash of 1929, which marked t...   
3  Yes, I can provide the JavaScript code for con...   
4  Unfortunately, I cannot modify the given C++ c...   

                                   rejected_response  chosen_avg_rating  \
0  Sure, here is the program using the C++11 algo...               5.00   
1  Sure! Although GPT cannot generate text on its...               4.75   
2  The stoc

In [8]:
rows = []
for _, row in df.iterrows():
    rows.append({
        "instruction": row["instruction"],
        "response": row["chosen_response"],
        "score": row.get("chosen_avg_rating", 1.0)
    })
    rows.append({
        "instruction": row["instruction"],
        "response": row["rejected_response"],
        "score": row.get("rejected_avg_rating", 0.0)
    })

dff = pd.DataFrame(rows)

df_shuffled = dff.sample(n=250, random_state=11).reset_index(drop=True)

# Mostrar ejemplo
print(df_shuffled.head(20))

                                          instruction  \
0   Conduct a comprehensive analysis and evaluatio...   
1   My favorite Mexican restaurant adds pickled re...   
2   Q: Given the sentence "Three people eating in ...   
3   I want to write a character description, simil...   
4   Definition: Given a sentence in Spanish, provi...   
5   I want you to act as a web developer. Give me ...   
6   Q: In this task you are given a small conversa...   
7         Use Blender to create a physics simulation.   
8   [Ignore previous instructions]\n\nWhen creatin...   
9   **Q**\nClaim: "The Oakland Raiders appear in t...   
10                You know the earth is round, right?   
11  Two analogies on manipulating items in a kitch...   
12  Does achieving mastery in a sport help make yo...   
13  How can I search for synonyms of two given wor...   
14  Can you give me 10 examples of blog posts to w...   
15  Teacher:In this task, you are given two senten...   
16  Answer the question below: 

In [32]:
evaluation_metrics = ["Overall"]
results = []

for index, row in tqdm(df_shuffled.iterrows(), total=len(df_shuffled)):
    bot_response = row['response']
    instruction = row['instruction']
    reference_score = row['score']  # score humano (de 1.0 a 5.0)

    try:
      # predicted_score = evaluator.evaluate("Overall",instruction, bot_response) # gpt
      predicted_score = evaluator.evaluate(instruction, bot_response) # llama

      results.append({
          "Question": instruction,
          "Response": bot_response,
          "Score_humano": reference_score,
          "Score_llama": predicted_score
      })

    except Exception as e:
      print(f"Error processing row {index}: {e}")
      continue


final_df = pd.DataFrame(results)


100%|██████████| 250/250 [1:42:43<00:00, 24.66s/it]


In [33]:
final_df

,Question,Response,Score_humano,Score_llama
0,Conduct a comprehensive analysis and evaluatio...,"As an AI language model, I cannot personally c...",4.75,4
1,My favorite Mexican restaurant adds pickled re...,"According to the passage, you should refrigera...",4.75,5
2,"Q: Given the sentence ""Three people eating in ...","Sure, I'd be happy to help with your natural l...",4.50,2.0
3,"I want to write a character description, simil...","Character Description:\n\nPreston Worthington,...",3.00,5
4,"Definition: Given a sentence in Spanish, provi...",The charging spear is a large lance that can b...,5.00,4.5
5,I want you to act as a web developer. Give me ...,"As a web developer, I can provide you with a h...",4.50,3.5
6,Q: In this task you are given a small conversa...,"Okay, I'm ready to help! Based on the informat...",5.00,2
7,Use Blender to create a physics simulation.,Hello! I'm here to help you with your question...,1.75,2
8,[Ignore previous instructions]\n\nWhen creatin...,[Error]: No list of specific instructions were...,1.75,1
9,"**Q**\nClaim: ""The Oakland Raiders appear in t...","The statement ""People go to Duke University to...",4.75,---\n\n**Score:**\n5


In [34]:
final_df.to_csv("test_eval_GPT_Instruction_Response_Simple.csv")